In [6]:
from scipy.io import arff
import pandas as pd
import numpy as np
from itertools import combinations
from mlxtend.frequent_patterns import apriori, association_rules



In [7]:
def load_arff(path):    
    data = arff.loadarff(path)
    df = pd.DataFrame(data[0])
    for col, dtype in df.dtypes.items():
        if dtype == object:  # Only process object columns.
            df[col] = df[col].str.decode('utf-8')
    return df

In [8]:
def discretize(df):
    for col, dtype in df.dtypes.items():
        if dtype == float or dtype == int:
            df[col] = pd.cut(df[col], 4,precision=0)
    return df


In [9]:
df = load_arff("./../profdata/weather.numeric.arff")
df = discretize(df)
df

,outlook,temperature,humidity,windy,play
0,sunny,"(80.0, 85.0]","(80.0, 88.0]",FALSE,no
1,sunny,"(80.0, 85.0]","(88.0, 96.0]",TRUE,no
2,overcast,"(80.0, 85.0]","(80.0, 88.0]",FALSE,yes
3,rainy,"(69.0, 74.0]","(88.0, 96.0]",FALSE,yes
4,rainy,"(64.0, 69.0]","(73.0, 80.0]",FALSE,yes
5,rainy,"(64.0, 69.0]","(65.0, 73.0]",TRUE,no
6,overcast,"(64.0, 69.0]","(65.0, 73.0]",TRUE,yes
7,sunny,"(69.0, 74.0]","(88.0, 96.0]",FALSE,no
8,sunny,"(64.0, 69.0]","(65.0, 73.0]",FALSE,yes
9,rainy,"(74.0, 80.0]","(73.0, 80.0]",FALSE,yes


In [10]:
print([x for x in df.dtypes.items()])

[('outlook', dtype('O')), ('temperature', CategoricalDtype(categories=[(64.0, 69.0], (69.0, 74.0], (74.0, 80.0], (80.0, 85.0]], ordered=True)), ('humidity', CategoricalDtype(categories=[(65.0, 73.0], (73.0, 80.0], (80.0, 88.0], (88.0, 96.0]], ordered=True)), ('windy', dtype('O')), ('play', dtype('O'))]


In [11]:
df = pd.get_dummies(df,dtype=bool,sparse=True)
df

,outlook_overcast,outlook_rainy,outlook_sunny,"temperature_(64.0, 69.0]","temperature_(69.0, 74.0]","temperature_(74.0, 80.0]","temperature_(80.0, 85.0]","humidity_(65.0, 73.0]","humidity_(73.0, 80.0]","humidity_(80.0, 88.0]","humidity_(88.0, 96.0]",windy_FALSE,windy_TRUE,play_no,play_yes
0,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False
1,False,False,True,False,False,False,True,False,False,False,True,False,True,True,False
2,True,False,False,False,False,False,True,False,False,True,False,True,False,False,True
3,False,True,False,False,True,False,False,False,False,False,True,True,False,False,True
4,False,True,False,True,False,False,False,False,True,False,False,True,False,False,True
5,False,True,False,True,False,False,False,True,False,False,False,False,True,True,False
6,True,False,False,True,False,False,False,True,False,False,False,False,True,False,True
7,False,False,True,False,True,False,False,False,False,False,True,True,False,True,False
8,False,False,True,True,False,False,False,True,False,False,False,True,False,False,True
9,False,True,False,False,False,True,False,False,True,False,False,True,False,False,True


In [13]:
columns = [11,14]

sup_playyes_windyfalse = df.iloc[:,columns].prod(1).mean()

print(sup_playyes_windyfalse)

0.42857142857142855


In [ ]:
def generate_new_combinations1(old_combinations):
    items_types_in_previous_step = np.unique(old_combinations.flatten())
    for old_combination in old_combinations:
        max_combination = old_combination[-1]
        mask = items_types_in_previous_step > max_combination
        valid_items = items_types_in_previous_step[mask]
        old_tuple = tuple(old_combination)
        for item in valid_items:
            yield from old_tuple
            yield item


def apriori1(df, min_support=0.5, use_colnames=True):
    def _support(_x):
        return _x.mean(0).reshape(-1)

    if min_support <= 0.0:
        raise ValueError(
            "`min_support` doit etre positive "
            "il doit etre compris dans l'interval `(0, 1]`. "
            "minsup %s." % min_support
        )
    
    X = df.values
    support = _support(X)
    ary_col_idx = np.arange(X.shape[1])
    support_dict = {1: support[support >= min_support]}
    itemset_dict = {1: ary_col_idx[support >= min_support].reshape(-1, 1)}
    max_itemset = 1

    while True :
        next_max_itemset = max_itemset + 1
        combin = generate_new_combinations(itemset_dict[max_itemset])
        combin = np.fromiter(combin, dtype=int)
        combin = combin.reshape(-1, next_max_itemset)
        
        if combin.size == 0:
            break

        _bools = np.all(X[:, combin], axis=2)
        support = _support(np.array(_bools))
        _mask = (support >= min_support).reshape(-1)
        
        if any(_mask):
            itemset_dict[next_max_itemset] = np.array(combin[_mask])
            support_dict[next_max_itemset] = np.array(support[_mask])
            max_itemset = next_max_itemset
        else:
            # Exit condition
            break
    all_res = []
    for k in sorted(itemset_dict):
        support = pd.Series(support_dict[k])
        itemsets = pd.Series([frozenset(i) for i in itemset_dict[k]], dtype="object")

        res = pd.concat((support, itemsets), axis=1)
        all_res.append(res)

    res_df = pd.concat(all_res)
    res_df.columns = ["support", "itemsets"]

    res_df = res_df.reset_index(drop=True)

    return res_df


In [ ]:
itemsets = apriori(df, min_support=0.4, use_colnames=True)
itemsets

,support,itemsets
0,0.571429,(windy_FALSE)
1,0.428571,(windy_TRUE)
2,0.642857,(play_yes)
3,0.428571,"(play_yes, windy_FALSE)"


In [ ]:
def association_rules1(df, metric="confidence", min_threshold=0.8, support_only=False):
    
    # metrics for association rules
    metric_dict = {
        "antecedent support": lambda _, sA, __: sA,
        "consequent support": lambda _, __, sC: sC,
        "support": lambda sAC, _, __: sAC,
        "confidence": lambda sAC, sA, _: sAC / sA,
        "lift": lambda sAC, sA, sC: metric_dict["confidence"](sAC, sA, sC) / sC,
        "leverage": lambda sAC, sA, sC: metric_dict["support"](sAC, sA, sC) - sA * sC,
    }

    columns_ordered = [
        "antecedent support",
        "consequent support",
        "support",
        "confidence",
        "lift",
        "leverage",
    ]

    # check for metric compliance
    if support_only:
        metric = "support"
    else:
        if metric not in metric_dict.keys():
            raise ValueError(
                "Metric must be 'confidence' or 'lift', got '{}'".format(metric)
            )

    # get dict of {frequent itemset} -> support
    keys = df["itemsets"].values
    values = df["support"].values
    frozenset_vect = np.vectorize(lambda x: frozenset(x))
    frequent_items_dict = dict(zip(frozenset_vect(keys), values))

    # prepare buckets to collect frequent rules
    rule_antecedents = []
    rule_consequents = []
    rule_supports = []

    # iterate over all frequent itemsets
    for k in frequent_items_dict.keys():
        sAC = frequent_items_dict[k]
        # to find all possible combinations
        for idx in range(len(k) - 1, 0, -1):
            # of antecedent and consequent
            for c in combinations(k, r=idx):
                antecedent = frozenset(c)
                consequent = k.difference(antecedent)

                if support_only:
                    # support doesn't need these,
                    # hence, placeholders should suffice
                    sA = None
                    sC = None

                else:
                    try:
                        sA = frequent_items_dict[antecedent]
                        sC = frequent_items_dict[consequent]
                    except KeyError as e:
                        s = (
                            str(e) + "You are likely getting this error"
                            " because the DataFrame is missing "
                            " antecedent and/or consequent "
                            " information."
                            " You can try using the "
                            " `support_only=True` option"
                        )
                        raise KeyError(s)
                    # check for the threshold

                score = metric_dict[metric](sAC, sA, sC)
                if score >= min_threshold:
                    rule_antecedents.append(antecedent)
                    rule_consequents.append(consequent)
                    rule_supports.append([sAC, sA, sC])

    # check if frequent rule was generated
    if not rule_supports:
        return pd.DataFrame(columns=["antecedents", "consequents"] + columns_ordered)

    else:
        # generate metrics
        rule_supports = np.array(rule_supports).T.astype(float)
        df_res = pd.DataFrame(
            data=list(zip(rule_antecedents, rule_consequents)),
            columns=["antecedents", "consequents"],
        )

        if support_only:
            sAC = rule_supports[0]
            for m in columns_ordered:
                df_res[m] = np.nan
            df_res["support"] = sAC

        else:
            sAC = rule_supports[0]
            sA = rule_supports[1]
            sC = rule_supports[2]
            for m in columns_ordered:
                df_res[m] = metric_dict[m](sAC, sA, sC)

        return df_res

In [ ]:
association_rules(itemsets, metric="confidence", min_threshold=0.4, support_only=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(play_yes),(windy_FALSE),0.642857,0.571429,0.428571,0.666667,1.166667,0.061224,1.285714
1,(windy_FALSE),(play_yes),0.571429,0.642857,0.428571,0.750000,1.166667,0.061224,1.428571
